# Working with Data

The intent of this tutorial is to help familiarize yourself with browsing for data that will be used along with an application to generate data by submitting a job. Job submission will be covered in the next tutorial.

In [ ]:
import requests
import getpass
import json
from IPython.display import JSON

First we need some pre-defined environment variables

In [ ]:
# This portion of the code is env specific for Dev, Test, Ops, etc. 
env = {
    "clientId":"71894molftjtie4dvvkbjeard0",
    "url":"https://58nbcawrvb.execute-api.us-west-2.amazonaws.com/test/"
      }

# The auth_json is template for authorizing with AWS Cognito for a token that can be used for calls to the data service.
auth_json = '''{
     "AuthParameters" : {
        "USERNAME" : "",
        "PASSWORD" : ""
     },
     "AuthFlow" : "USER_PASSWORD_AUTH",
     "ClientId" : ""
  }'''

### Authentication Code

The below method is a helper function for getting a token for accessing unity services

In [ ]:
# This method is used for taking a username and password and client ID and fetching a cognito token
def get_token(username, password, clientID):
    aj = json.loads(auth_json)
    aj['AuthParameters']['USERNAME'] = username
    aj['AuthParameters']['PASSWORD'] = password
    aj['ClientId'] = clientID 
    token = None
    try:
        response = requests.post('https://cognito-idp.us-west-2.amazonaws.com', headers={"Content-Type":"application/x-amz-json-1.1", "X-Amz-Target":"AWSCognitoIdentityProviderService.InitiateAuth"}, json=aj)
        token = response.json()['AuthenticationResult']['AccessToken']
    except:
        print("Error, check username and password and try again.")
    return token

### Ask the User for their Unity username, password

These are required to get the token (described above) to connect to the data services.

In [ ]:
print("Please enter your username...")
user_name = input()

print("Please enter your password...")
password = getpass.getpass()

In [ ]:
token = get_token(user_name, password, env['clientId'])

## List Available Collections in the Unity System

In [ ]:
url = env['url'] + "am-uds-dapa/collections"
response = requests.get(url, headers={"Authorization": "Bearer " + token})

#print(json.dumps(response.json()))
JSON(response.json())

## Given a collection (above), List the files within that collection

In [ ]:

url = env['url'] + "am-uds-dapa/collections/L0_SNPP_ATMS_SCIENCE___1/items"
params = []
params.append(("limit", 100))

response = requests.get(url, headers={"Authorization": "Bearer " + token}, params=params)
JSON(response.json())


## Filter the results above by time

In [ ]:
url = env['url'] + "am-uds-dapa/collections/L0_SNPP_ATMS_SCIENCE___1/items"
# the datetime,limit, and offset are included due to a current bug in the API Gatway setting these values to 'none'.
# Example date/time params

params = []
#add a datetime to your request
params.append(("datetime", "2016-11-01T00:00:00Z/2016-11-01T12:31:12Z"))

# limit - how many results to return in a single request
params.append(("limit", 100))

#offset - the number of results to offset, for paging
#params.append(("offset", 100))

#response = requests.get('https://58nbcawrvb.execute-api.us-west-2.amazonaws.com/test/', headers={"Authorization": "Bearer " + token})
response = requests.get(url, headers={"Authorization": "Bearer " + token}, params=params)

#print(json.dumps(response.json()))
JSON(response.json())
